# Team work: Handling of available data

For testing your hypothesis in the team work part of the course, different datasets are provided:

- [EODC STAC Catalog](https://services.eodc.eu/browser/#/v1/): Sentinel-1 and other datasets
- Data on JupyterHub:
    - Data used within the hands-on exercise (`~/shared/datasets/rs`): ALOS, ASCAT, soil moisture, ...
    - Austrian Datacube (`~/shared/datasets/fe/data`): Sentinel-1, Sentinel-2, Corine Land Cover, ...
- Additionally, you can bring in your own datasets or access other STAC catalogs.

In this notebook, examples are given how to access the different data sources through the JupyterHub. All shown methods aim to load a `xarray` object, which allows to use many predefined functions and offers a detailed [documentdation](https://docs.xarray.dev/). Besides that you can also work in QGIS, run local Python environments on you own PC or pick tools you are used to. Please be aware that your work needs to end up in a report and presentation at the end!

## STAC data access: Sentinel-1 from EODC

Here, a quick recap of [Unit 1]() of the course on how to access data available through STAC. For more details re-visit the corresponding notebook!

In [ ]:
import gc
import os

import xarray as xr
import numpy as np
from matplotlib import pyplot as plt

import pystac_client
from odc import stac as odc_stac

EODC offers a lot of datasets through STAC, but be aware that not all might be accessible. To get an overview, have a look at the collections:

In [ ]:
# collections provided by the EODC STAC Catalog
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1")
collections = eodc_catalog.get_collections()

max_length = max(len(collection.id) for collection in collections)
for collection in eodc_catalog.get_collections():
    print(f"{collection.id.ljust(max_length)}: {collection.title}")

We will now load **Sentinel-1** data for the area surrounding **Innsbruck** using **STAC**.  
First, we define a **spatial bounding box** covering the Innsbruck region, and apply a **temporal filter** spanning the entire month of **March 2022**.

In [ ]:
time_range = "2022-03-01/2022-03-31"
innsbruck_bbox = (11.070099, 47.148400, 11.729279, 47.380219)

Next, we query the **EODC STAC catalog** for items from the **SENTINEL1_SIG0_20M** collection that match these spatiotemporal constraints.

In [ ]:
s1_collection_id = "SENTINEL1_SIG0_20M"
search = eodc_catalog.search(
    collections=s1_collection_id,
    bbox=innsbruck_bbox,
    datetime=time_range,
)
s1_items = search.item_collection()
len(s1_items)

We use the **`odc-stac`** package to load STAC-compliant items directly into an **xarray** dataset.  
For more details, refer to the [official documentation](https://odc-stac.readthedocs.io/en/latest/).

Specifically, we'll load the **Sentinel-1 SIG0** data we retrieved earlier as a three-dimensional datacube with dimensions **(time, lat, lon)**, and extract the **VV** polarization band.

In [ ]:
bands = "VV"
chunks = {"time": 1, "latitude": 500, "longitude": 500}

sig0_dc = odc_stac.load(
    s1_items,
    bands=bands,
    crs="EPSG:27704",
    resolution=20,
    bbox=innsbruck_bbox,
    chunks=chunks,
    resampling="bilinear",
)
sig0_dc

To reduce storage requirements, this data is stored as **int16 arrays** together with a **scaling factor** that converts the integer values to floating-point reflectance values on demand.  
To visualize or process the data correctly, we need to read this scaling factor from the dataset's metadata and apply it.

Finally, we'll also inspect the metadata to find the **nodata flag** (the value used to mark missing data) and filter out those empty observations before proceeding.

In [ ]:

scale = s1_items[0].assets["VV"].extra_fields.get("raster:bands")[0]["scale"]
nodata = s1_items[0].assets["VV"].extra_fields.get("raster:bands")[0]["nodata"]
sig0_dc = sig0_dc.where(sig0_dc != nodata) / scale
sig0_dc = sig0_dc.dropna(dim="time")
sig0_dc.VV

We then compute the mean over the timeslices that remained, and plot it with xarray's plotting function, which provides us with some quality-of-life features like automatically adding axis labels and a legend for the colormap.

In [ ]:
sig0_mean = sig0_dc.mean(dim="time", skipna=True)
sig0_mean.VV.plot(robust=True, cmap="Greys_r")

Finally, we deallocate the memory for the data we just loaded and manually run python's garbage collector.

In [ ]:
del sig0_dc
gc.collect()

## STAC data access: Sentinel-2 from CDSE

There are several services that provide **Earth Observation (EO)** data through **STAC APIs**.  
One of them is the **[Copernicus Data Space Ecosystem (CDSE)](https://dataspace.copernicus.eu/)**. In this section, we will access and manipulate Sentinel-2 optical data provided by CDSE.

The CDSE STAC catalog can be accessed via the  
[CDSE STAC Browser](https://browser.stac.dataspace.copernicus.eu/?language=en). Comprehensive documentation is available on the [CDSE website](https://dataspace.copernicus.eu/). Unlike the **EODC** STAC API, the **CDSE** API requires users to register and generate **S3 API keys**.  
This process is straightforward and described in the [official guide](https://documentation.dataspace.copernicus.eu/APIs/S3.html).

Once you have created your keys, copy both the **Access Key** and **Secret Access Key** into a text file named `keys.txt`, placed in the same directory as this notebook.  
Running the cell below will then configure the necessary environment variables, enabling access to the **CDSE STAC API**.


In [ ]:
with open("keys.txt", "r") as f:
    access_key, secret_access_key = [line.strip() for line in f.readlines()]

os.environ["AWS_ACCESS_KEY_ID"] = access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_access_key
os.environ["AWS_S3_ENDPOINT"] = "eodata.dataspace.copernicus.eu"
os.environ["AWS_VIRTUAL_HOSTING"] = "FALSE"


Now we simply follow a similar procedure to the one we employed for the EODC STAC, listing all the collections available in the CDSE catalog.

In [ ]:
# collections provided by the CDSE STAC Catalog
cdse_catalog = pystac_client.Client.open("https://stac.dataspace.copernicus.eu/v1")

cdse_collections = cdse_catalog.get_collections()
max_length = max(len(collection.id) for collection in cdse_collections)

for collection in cdse_catalog.get_collections():
    print(f"{collection.id.ljust(max_length)}: {collection.title}")

Again, just like in the Sentinel-1 Section, we define a spatiotemporal filter around the Innsbruck region and covering March 2022. We then query the CDSE catalog for Sentinel-2 Level 2A optical data that fulfills our requirements.

In [ ]:
time_range = "2022-03-01/2022-03-31"
innsbruck_bbox = (11.070099, 47.148400, 11.729279, 47.380219)

s2_collection_id = "sentinel-2-l2a"
search = cdse_catalog.search(
    collections=s2_collection_id,
    bbox=innsbruck_bbox,
    datetime=time_range,
)
s2_items = search.item_collection()
len(s2_items)

This time, we query an xarray dataset with the bands ["B02_10m","B03_10m","B04_10m"], which correspond to blue, green and red, respectively. 

In [ ]:
chunks = {"time": 1, "latitude": 500, "longitude": 500}


bands=["B02_10m","B03_10m","B04_10m"]

s2_dc = odc_stac.load(
    s2_items,
    bands=bands,
    crs="EPSG:27704",
    resolution=20,
    bbox=innsbruck_bbox,
    chunks=chunks,
    resampling="bilinear",
)

s2_dc

Next, we'll visualize the first observation from our search results as a **true-color RGB image**.

To do this, we first create a new **xarray `DataArray`** that stacks the three visible bands (red, green, and blue) along a new `band` dimension using `xr.concat`.  
This gives us an array with dimensions **(band, lat, lon)**, where each band corresponds to one color channel.

We then scale the data values to the **[0, 1] range** by dividing by the appropriate factor, ensuring the image displays correctly when plotted.

Finally, we use **`xarray.plot.imshow()`** with the `rgb="band"` argument, which tells xarray to interpret the `band` dimension as RGB channels and render a color image automatically.


In [ ]:
first_date = str(np.datetime_as_string(s2_dc.time[0], unit='D'))
s2_data_first_date = s2_dc.isel(time=0)

rgb_da = xr.concat(
    [s2_data_first_date["B04_10m"],  # red
     s2_data_first_date["B03_10m"],  # green
     s2_data_first_date["B02_10m"]], # blue
    dim="band"
).assign_coords(band=["R", "G", "B"])

rgb_da = rgb_da.astype("float32") / 10000.0
rgb_da = rgb_da.clip(0.0, 1.0)

rgb_da.plot.imshow(rgb="band")
plt.title(f"Sentinel-2 L2A RGB, {first_date}")
plt.show()